In [60]:
import numpy as np
from time import time
import torch

In [61]:
img = np.arange(256)[:,np.newaxis]@np.ones(256)[np.newaxis,:]
kernel = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])

In [62]:
x = kernel.shape[0]-1; y = kernel.shape[1]-1
p_img = np.pad(img,((x,x),(y,y)))
a = (p_img.shape[0]-kernel.shape[0])//2
b = int(np.ceil((p_img.shape[0]-kernel.shape[0])/2))


p_img = torch.Tensor(p_img)
##Note: You can take the fft of the kernel and pad it ahead of time while testing since it is a constant.
padded_kernel = torch.nn.functional.pad(torch.fft.fft2(torch.Tensor(kernel)),(b,a,b,a))

In [63]:
start_time = time()
out_fft = (torch.fft.fft2(p_img) * padded_kernel)[x:-x,y:-y]
print(time() - start_time)

0.0016040802001953125


In [64]:
img = torch.Tensor(img)
kernel = torch.Tensor(kernel)

img = torch.reshape(img,(1,1,img.shape[0],img.shape[1]))
kernel = torch.reshape(kernel,(1,1,kernel.shape[0],kernel.shape[1]))

In [65]:
start_time = time()
out_conv = torch.nn.functional.conv2d(img,kernel,padding=(1,1))
print(time() - start_time)

0.0033311843872070312


In [66]:
out_fft == out_conv

tensor([[[[False,  True,  True,  ...,  True,  True, False],
          [False,  True,  True,  ...,  True,  True, False],
          [False,  True,  True,  ...,  True,  True, False],
          ...,
          [False,  True,  True,  ...,  True,  True, False],
          [False,  True,  True,  ...,  True,  True, False],
          [False,  True,  True,  ...,  True,  True, False]]]])

In [68]:
## Still working on de-bugging why first column and last column are not equal
out_fft[:,1:-1] == out_conv[0,0,:,1:-1]

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])